In [1]:
# on server: 'screen' ,then start script
# use 'strg+a d' to return to terminal
# use 'screen -r' to return to screen
import numpy as np
import json
import os

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.layers import Input,Dense,LSTM,GRU
from keras.layers import Bidirectional
from keras.utils import plot_model
# os.environ['CUDA_VISIBLE_DEVICES']='0'

import process_data as pd

Using TensorFlow backend.


In [2]:
# https://towardsdatascience.com/nlp-sequence-to-sequence-networks-part-1-processing-text-data-d141a5643b72
path='testpath/'
# rnn parameters
hidden_size = 100 #100 is the standard
batch_size = 512 #for the training on the GPU this to be has to very large, otherwise the GPU is used very inefficiently
epochs = 100

size=1000

#glove embedding parameters
glove_dir = '../glove/glove.6B.100d.txt'
embedding_dim = 100

In [3]:
#open SQuAD-dataset and extract the relevant data from the json-file
#to a easier readable/accessible dictionary
with open('SQuAD/train-v2.0.json') as file:
    train=json.load(file)
train_context=[]
train_question=[]
train_answer=[]
train_new={'context':train_context,'question':train_question,'answer':train_answer}
for j,data in enumerate(train['data']):
    for i,paragraph in enumerate(data['paragraphs']):
        context=paragraph['context']
        for qas in paragraph['qas']:
            #create a dataset with only the answerable questions
            #add a bos and eos token to the target
            if (qas['is_impossible']==False):
                a=context.lower()
                b=qas['question'].lower()
                c=qas['answers'][0]['text'].lower()
                
                train_new['context'].append('\t'+a+'\n')
                train_new['question'].append('\t'+b+'\n')
                train_new['answer'].append('\t'+c+'\n')
print(len(train_new['context']))
print(len(train_new['question']))
print(len(train_new['answer']))

86821
86821
86821


In [4]:
context=train_new['context'][:size]
question=train_new['question'][:size]
answer=train_new['answer'][:size]
data=[context,question,answer]
input_data=pd.process_data(data)

In [6]:
#FIX_ME: add glove download
# https://nlp.stanford.edu/projects/glove/
#get glove embeddings
embeddings_index = {}
f = open(glove_dir)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [8]:
#extract the glove-embedding to a matrix
context_embedding_matrix = np.zeros((input_data['len_vocab']['len_context_vocab'], embedding_dim))
for word, i in input_data['token_to_int']['context_token_to_int'].items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        context_embedding_matrix[i] = embedding_vector

question_embedding_matrix = np.zeros((input_data['len_vocab']['len_question_vocab'], embedding_dim))
for word, i in input_data['token_to_int']['question_token_to_int'].items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        question_embedding_matrix[i] = embedding_vector

answer_embedding_matrix = np.zeros((input_data['len_vocab']['len_answer_vocab'], embedding_dim))
for word, i in input_data['token_to_int']['answer_token_to_int'].items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        answer_embedding_matrix[i] = embedding_vector
print(np.shape(context_embedding_matrix),np.shape(question_embedding_matrix),np.shape(answer_embedding_matrix))

(3578, 100) (2033, 100) (955, 100)


In [11]:
# Define an input sequence and process it.
context_encoder_inputs = Input(shape=(None,))
context_x = Embedding(input_data['len_vocab']['len_context_vocab'], embedding_dim,weights=[context_embedding_matrix],
                      trainable=False)(context_encoder_inputs)
context_x, context_state_h, context_state_c = LSTM(embedding_dim,
                           return_state=True)(context_x)
context_encoder_states = [context_state_h, context_state_c]

question_encoder_inputs = Input(shape=(None,))
question_x = Embedding(input_data['len_vocab']['len_question_vocab'], embedding_dim,weights=[question_embedding_matrix],
                       trainable=False)(question_encoder_inputs)
question_x, question_state_h, question_state_c = LSTM(embedding_dim,
                           return_state=True)(question_x)
question_encoder_states = [question_state_h, question_state_c]

state_h=layers.Concatenate()([context_state_h,question_state_h])
state_c=layers.Concatenate()([context_state_c,question_state_c])
concat_states=[state_h,state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
x = Embedding(input_data['len_vocab']['len_answer_vocab'], embedding_dim,weights=[answer_embedding_matrix],trainable=False)(decoder_inputs)
x = LSTM(embedding_dim*2, return_sequences=True)(x, initial_state=concat_states)
decoder_outputs = Dense(input_data['len_vocab']['len_answer_vocab'], activation='softmax')(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([context_encoder_inputs,question_encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 100)    357800      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 100)    203300      input_6[0][0]                    
__________________________________________________________________________________________________
input_7 (I

In [ ]:
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
# plot_model(model, to_file=path+'/model.png')

In [ ]:
# # Note that `decoder_target_data` needs to be one-hot encoded,
# # rather than sequences of integers like `decoder_input_data`!

# model.fit([encoder_input_context,encoder_input_question, decoder_input_answer], decoder_target_answer,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.2)